In [24]:
import matplotlib.pyplot as plt
from database import scrape as sc
from evaluate import Evaluate
from predict import Predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
import pandas as pd
warnings.simplefilter(action="ignore")
plt.style.use("seaborn-v0_8-darkgrid")


In [37]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
df.columns

Index(['GAME_ORDER', 'YEAR', 'DATE', 'GAME_NUMBER', 'WEEK', 'PLAYER', 'AGE',
       'TEAM', 'OPPONENT', 'RESULT', 'RUSHING_ATT', 'RUSHING_YDS',
       'RUSHING_TD', 'RUSHING_1D', 'RUSHING_YBC', 'RUSHING_YBC_ATT',
       'RUSHING_YAC', 'RUSHING_YAC_ATT', 'RUSHING_BRKTKL', 'RUSHING_ATT_BR',
       'RECEIVING_TGT', 'RECEIVING_REC', 'RECEIVING_YDS', 'RECEIVING_TD',
       'RECEIVING_1D', 'RECEIVING_YBC', 'RECEIVING_YBC_R', 'RECEIVING_YAC',
       'RECEIVING_YAC_R', 'RECEIVING_ADOT', 'RECEIVING_BRKTKL',
       'RECEIVING_REC_BR', 'RECEIVING_DROP', 'RECEIVING_DROP_PCT',
       'RECEIVING_INT', 'RECEIVING_RAT', 'PASSING_CMP', 'PASSING_ATT',
       'PASSING_CMP_PCT', 'PASSING_YDS', 'PASSING_TD', 'PASSING_INT',
       'PASSING_RATE', 'PASSING_SK', 'PASSING_YDS1', 'PASSING_Y_A',
       'PASSING_AY_A', 'PPR'],
      dtype='object')

In [26]:
top_180_eval = [
    Evaluate(
        df=df, player=name, steps=17, regressor=RandomForestRegressor, lags=9
    ).eval()
    for name in list(set(df['PLAYER']))
]

df_top_180 = pd.concat(top_180_eval)

Not enough data. Adjusting lags for Sam LaPorta.
Not enough data. Adjusting steps for Breece Hall.
Not enough data. Adjusting steps for Ty Chandler.
Not enough data. Adjusting lags for Tyjae Spears.
Not enough data. Adjusting lags for Puka Nacua.
Not enough data. Adjusting steps for Kyren Williams.
Not enough data. Adjusting lags for Jayden Reed.
Not enough data. Adjusting lags for Jaleel McLaughlin.
Not enough data. Adjusting steps for Khalil Shakir.
Not enough data. Adjusting steps for Rashid Shaheed.
Not enough data. Adjusting steps for Jahan Dotson.
Not enough data. Adjusting lags for Dalton Kincaid.
Not enough data. Adjusting steps for Brock Purdy.
Not enough data. Adjusting lags for Jake Browning.
Not enough data. Adjusting lags for Dontayvion Wicks.
Not enough data. Adjusting lags for Michael Wilson.
Not enough data. Adjusting steps for Desmond Ridder.
Not enough data. Adjusting lags for Easton Stick.
Not enough data. Adjusting lags for Jaxon Smith-Njigba.
Not enough data. Adjus

In [27]:
df_top_180.sort_values('ACTUAL SEASON TOTAL', ascending=False)

,PLAYER,ACTUAL SEASON TOTAL,PREDICTED,SEASON MAPE,GAME MAPE,GAME RSME,GAMES PREDICTED,LAGS,REGRESSOR
0,Josh Allen,430.64,388.0392,0.098924,2.386837e-01,7.531729,17,9,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,Christian McCaffrey,412.20,380.5164,0.076865,2.809280e-01,9.372577,17,9,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,CeeDee Lamb,405.20,235.7780,0.418119,4.271846e-01,14.948801,17,9,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,Jalen Hurts,396.82,397.7674,0.002387,6.245587e-01,7.287698,17,9,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,Tyreek Hill,382.70,333.8930,0.127533,6.353078e-01,11.814355,17,9,"(DecisionTreeRegressor(max_features=1.0, rando..."
...,...,...,...,...,...,...,...,...,...
0,Michael Wilson,35.40,39.2600,0.109040,1.383326e+16,6.565478,5,5,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,Jaleel McLaughlin,33.00,27.8720,0.155394,2.534716e+00,3.862453,7,7,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,Easton Stick,29.92,26.7768,0.105053,1.910506e-01,3.439989,2,2,"(DecisionTreeRegressor(max_features=1.0, rando..."
0,Tommy DeVito,13.38,48.2810,2.608445,2.216161e+16,12.359505,3,3,"(DecisionTreeRegressor(max_features=1.0, rando..."


In [28]:
np.mean(df_top_180['SEASON MAPE']), np.mean(df_top_180['GAME RSME']), np.mean(df_top_180['GAME MAPE'])

(0.2915313964992749, 8.193352464110758, 1275231635640061.8)

In [29]:
np.sqrt(mean_squared_error(df_top_180['ACTUAL SEASON TOTAL'], df_top_180['PREDICTED']))

56.690422774080844

In [30]:
top_180_pred = [
Predict(
    df=df, player=i, steps=17, regressor=GradientBoostingRegressor, lags=9
).predict_season() for i in list(set(df['PLAYER']))
]

clean_data = [row for row in top_180_pred if row is not None]

Jake Browning has insufficient data. Only 9 games on record
Easton Stick has insufficient data. Only 6 games on record
Tommy DeVito has insufficient data. Only 9 games on record
Anthony Richardson has insufficient data. Only 4 games on record
Will Levis has insufficient data. Only 9 games on record


In [52]:
df_preds = pd.DataFrame(clean_data, columns=['PLAYER', 'PROJECTED POINTS'])

In [53]:
df_2023 = df.loc[df['YEAR']==2023]
df_players_2023 = df_2023.groupby(['PLAYER', 'TEAM'])['PPR'].sum().reset_index()
df_players_2023.rename(columns={'PPR':'PPR_2023'}, inplace=True)

In [54]:
df_adp = pd.read_csv('ADP_2024.csv')
df_adp['Position'] = df_adp['Consensus Pos'].str[:2]
df_adp['Target Round'] = (df_adp['Consensus'] // 10).astype(int) + 1

In [56]:
df_adp

,Consensus,Player,Consensus Pos,Team/Bye,BestBall10s,CBS,DraftKings,Drafters,ESPN,MyFantasyLeague,NFFC,NFLcom,RT Sports,Sleeper,Underdog,Yahoo!,Position,Target Round
0,1,Christian McCaffrey,RB1,SF\n ...,1.0,1,1,1,1,1.0,1.0,1,1.0,1,1,1.0,RB,1
1,2,CeeDee Lamb,WR1,DAL\n ...,2.0,2,2,2,3,3.0,2.0,2,2.0,2,2,3.0,WR,1
2,3,Tyreek Hill,WR2,MIA\n ...,3.0,3,3,3,2,2.0,3.0,3,3.0,3,3,2.0,WR,1
3,4,Ja'Marr Chase,WR3,CIN\n ...,5.0,6,4,4,6,7.0,6.0,4,8.0,6,4,6.0,WR,1
4,5,Breece Hall,RB2,NYJ\n ...,4.0,4,8,8,4,5.0,5.0,7,4.0,8,8,4.0,RB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,454,Boston Scott,RB109,FA,NaN,-,-,-,-,NaN,NaN,-,NaN,-,317,NaN,RB,46
454,455,Durham Smythe,TE55,MIA\n ...,NaN,-,-,-,-,NaN,300.0,-,NaN,-,-,NaN,TE,46
455,456,J.J. Taylor,RB110,HOU\n ...,NaN,-,-,-,249,NaN,NaN,-,NaN,-,-,NaN,RB,46
456,457,Sean Tucker,RB111,TB\n ...,NaN,-,-,-,-,NaN,NaN,-,NaN,-,319,NaN,RB,46


In [58]:
df_merge1 = pd.merge(df_players_2023, df_preds,  how='inner', on='PLAYER')

In [62]:
df_final = pd.merge(df_merge1, df_adp[['Player', 'Position', 'Consensus', 'Target Round']], left_on='PLAYER', right_on='Player', how='inner').drop(columns=['Player'])
df_final.sort_values(by='PROJECTED POINTS', ascending=False).head(25)

,PLAYER,TEAM,PPR_2023,PROJECTED POINTS,Position,Consensus,Target Round
22,CeeDee Lamb,DAL,405.20,476.543399,WR,2,1
112,Lamar Jackson,BAL,355.22,451.387628,QB,41,5
73,Jalen Hurts,PHI,396.82,420.078503,QB,36,4
111,Kyren Williams,LAR,257.00,418.311881,RB,17,2
122,Nico Collins,HOU,258.40,404.129407,WR,23,3
26,Christian McCaffrey,SFO,395.30,399.726826,RB,1,1
91,Josh Allen,BUF,430.64,399.088443,QB,29,3
17,Brock Purdy,SFO,321.60,376.471911,QB,94,10
99,Justin Herbert,LAC,248.16,373.904490,QB,127,13
8,Amon-Ra St. Brown,DET,332.90,368.903035,WR,8,1


In [67]:
ranked_values = {
    'QB': df_final[df_final['Position'] == 'QB'].nlargest(10, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'QB']) >= 10 else df_final[df_final['Position'] == 'QB']['PROJECTED POINTS'].min(),
    'TE': df_final[df_final['Position'] == 'TE'].nlargest(10, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'TE']) >= 10 else df_final[df_final['Position'] == 'TE']['PROJECTED POINTS'].min(),
    'WR': df_final[df_final['Position'] == 'WR'].nlargest(30, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'WR']) >= 30 else df_final[df_final['Position'] == 'WR']['PROJECTED POINTS'].min(),
    'RB': df_final[df_final['Position'] == 'RB'].nlargest(20, 'PROJECTED POINTS')['PROJECTED POINTS'].iloc[-1] if len(df_final[df_final['Position'] == 'RB']) >= 20 else df_final[df_final['Position'] == 'RB']['PPROJECTED POINTS'].min()
}

# Print the ranked PPR values for debugging
print("Ranked PPR values:")
print(ranked_values)

Ranked PPR values:
{'QB': 337.6109885148496, 'TE': 184.53266589377128, 'WR': 210.5260954459529, 'RB': 226.92921916285178}


In [70]:
def VBD(row):
    category = row['Position']
    ranked_ppr = ranked_values.get(category, 0) 
    return row['PROJECTED POINTS'] - ranked_ppr

# Apply the function to each row
df_final['VBD'] = df_final.apply(VBD, axis=1)


In [73]:
df_final['My Ranking'] = df_final['VBD'].rank(ascending=False, method='min').astype(int)
df_final.sort_values(by='VBD', ascending=False)

,PLAYER,TEAM,PPR_2023,PROJECTED POINTS,Position,Consensus,Target Round,VBD,My Ranking
22,CeeDee Lamb,DAL,405.20,476.543399,WR,2,1,266.017303,1
122,Nico Collins,HOU,258.40,404.129407,WR,23,3,193.603311,2
111,Kyren Williams,LAR,257.00,418.311881,RB,17,2,191.382661,3
26,Christian McCaffrey,SFO,395.30,399.726826,RB,1,1,172.797607,4
8,Amon-Ra St. Brown,DET,332.90,368.903035,WR,8,1,158.376939,5
...,...,...,...,...,...,...,...,...,...
11,Bailey Zappe,NWE,82.86,129.892756,QB,445,45,-207.718232,160
135,Sam Howell,WAS,299.54,123.098661,QB,346,35,-214.512328,161
39,Dameon Pierce,HOU,76.70,0.632474,RB,192,20,-226.296746,162
158,Tyrod Taylor,NYG,79.82,91.158190,QB,441,45,-246.452798,163
